In [1]:
import pandas as pd


train_data = pd.read_csv("data\main\df_train_b1.csv", index_col=0)
test_data = pd.read_csv('data\main\df_test_b1.csv', index_col=0)

In [2]:
if True:

    import pandas as pd
    temp_df = pd.read_csv("./data/detail/df_loan1_b1.csv")

    def chuli_loan(df, colname, ways):
        cust_id = list(set(df['cust_id']))
        res_df = pd.DataFrame({'cust_id': cust_id})
        try:
            temp_train = pd.read_csv('./data/df_train.csv')
            temp_test = pd.read_csv('./data/df_test.csv')
            temp = list(temp_test.columns)
        except:
            temp = []

        ways_1 = []

        global train_data, test_data

        for i in ways:
            if (colname+'_'+i) not in temp:
                ways_1.append(i)
                res_df[colname+'_'+i] = None
            else:
                print(colname+'_'+i+'已存在')
                train_data = pd.merge(train_data, temp_train[['cust_id', colname+'_'+i]], on="cust_id",
                                      how="left", suffixes=("", "_y"))
                test_data = pd.merge(test_data, temp_test[['cust_id', colname+'_'+i]], on="cust_id",
                                     how="left", suffixes=("", "_y"))
        ways = ways_1
        if len(ways) == 0:
            return res_df
        if 'mean' in ways:
            res_df[colname+'_mean'] = df.groupby('cust_id')[colname].mean()
        if 'sum' in ways:
            res_df[colname+'_sum'] = df.groupby('cust_id')[colname].sum()
        if 'count' in ways:
            res_df[colname+'_count'] = df.groupby('cust_id')[colname].count()
        if 'std' in ways:
            res_df[colname+'_std'] = df.groupby('cust_id')[colname].std()
        if 'diff_std' in ways:
            res_df[colname+'_diff_std'] = df.groupby('cust_id')[colname].apply(
                lambda x: x.sort_values(ascending=True).diff()[2:].std())
        return res_df

    for colname in ['loan1_2', 'loan1_3', 'loan1_7', 'loan1_9']:
        df = chuli_loan(temp_df, colname, ('mean', 'sum'))
        train_data = pd.merge(train_data, df, on="cust_id",
                              how="left", suffixes=("", "_y"))
        test_data = pd.merge(test_data, df, on="cust_id",   
                             how="left", suffixes=("", "_y"))
    for colname in ['loan1_2','loan1_3']:
        df = chuli_loan(temp_df,colname , ('count',))
        train_data = pd.merge(train_data, df, on="cust_id",
                                how="left", suffixes=("", "_y"))
        test_data = pd.merge(test_data, df, on="cust_id",
                                how="left", suffixes=("", "_y"))                    

loan1_2_mean已存在
loan1_2_sum已存在
loan1_3_mean已存在
loan1_3_sum已存在
loan1_7_mean已存在
loan1_7_sum已存在
loan1_9_mean已存在
loan1_9_sum已存在
loan1_2_count已存在
loan1_3_count已存在


In [3]:
df_train=train_data
df_test=test_data

In [4]:
df_train.drop(["loan1_17",'loan1_31'],axis=1,inplace=True)
df_test.drop(["loan1_17",'loan1_31'],axis=1,inplace=True)

In [5]:
def delnan(df):
    df = df.dropna(how="all", axis=1)  # 删除全是空值的列
    df = df.fillna(-99)
    return df

df_train = delnan(train_data)
df_test = delnan(test_data)

In [6]:
full_col_name = list(df_train.columns)
basic_col = [i for i in full_col_name if "basic" in i]
loan_col = [i for i in full_col_name if "loan" in i]
overdue_col = [i for i in full_col_name if "overdue" in i]
query_col = [i for i in full_col_name if "query" in i]
judicial_col = [i for i in full_col_name if "judicial" in i]
date_col = [i for i in full_col_name if "date" in i]

In [7]:
x_col = basic_col+['is_judicial']+loan_col+query_col+overdue_col+date_col
x_col=[i for i in x_col if i in full_col_name]

In [8]:
print(x_col)

['basic_1', 'basic_2', 'basic_3', 'basic_4', 'basic_5', 'basic_6', 'basic_7', 'basic_8', 'basic_9', 'basic_10', 'basic_11', 'basic_12', 'basic_13', 'basic_14', 'basic_15', 'loan1_1', 'loan1_2', 'loan1_3', 'loan1_4', 'loan1_5', 'loan1_6', 'loan1_7', 'loan1_8', 'loan1_9', 'loan1_10', 'loan1_11', 'loan1_12', 'loan1_13', 'loan1_14', 'loan1_15', 'loan1_16', 'loan1_18', 'loan1_19', 'loan1_20', 'loan1_21', 'loan1_22', 'loan1_23', 'loan1_24', 'loan1_25', 'loan1_26', 'loan1_27', 'loan1_28', 'loan1_29', 'loan1_30', 'loan1_32', 'loan1_33', 'loan2_1', 'loan2_2', 'loan2_3', 'loan2_4', 'loan2_5', 'loan2_6', 'loan2_7', 'loan2_8', 'loan2_9', 'loan2_10', 'loan2_11', 'loan2_12', 'loan1_2_mean', 'loan1_2_sum', 'loan1_3_mean', 'loan1_3_sum', 'loan1_7_mean', 'loan1_7_sum', 'loan1_9_mean', 'loan1_9_sum', 'loan1_2_count', 'loan1_3_count', 'query_1', 'query_2', 'query_3', 'query_4', 'query_5', 'query_6', 'query_7', 'query_8', 'query_9', 'query_10', 'query_11', 'overdue_1', 'overdue_2', 'overdue_3', 'overdue_4

In [9]:
import copy
import xgboost as xgb
import random
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation


In [10]:
X = df_train[x_col]
Y = df_train['label']

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, Y, test_size=3000, stratify=Y, random_state=2022)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
x_perd = xgb.DMatrix(df_test[x_col])

evallist = [(dtrain,"train"),(dtest, 'eval')]
param = {
    'max_depth': 5,
    'eta': 0.008,
    'colsample_bytree': 0.5,
    'objective': 'binary:logistic',
    # 'verbosity':0,
    'scale_pos_weight': 1,
    'eval_metric': 'auc',
    'subsample': 0.55,
    'min_child_weight': 10,
    'gamma': 0.5,
    'seed': 666,
    'tree_method': 'gpu_hist',
    'gpu_id': 0,
}

bst = xgb.train(param, dtrain, 3000, evals=evallist,early_stopping_rounds=300)

[0]	train-auc:0.58742	eval-auc:0.52430
[1]	train-auc:0.58742	eval-auc:0.52430
[2]	train-auc:0.58788	eval-auc:0.53132
[3]	train-auc:0.62922	eval-auc:0.55017
[4]	train-auc:0.64611	eval-auc:0.55496
[5]	train-auc:0.64674	eval-auc:0.55089
[6]	train-auc:0.64760	eval-auc:0.55845
[7]	train-auc:0.64760	eval-auc:0.55845
[8]	train-auc:0.65065	eval-auc:0.56313
[9]	train-auc:0.65053	eval-auc:0.56089
[10]	train-auc:0.65055	eval-auc:0.55721
[11]	train-auc:0.65409	eval-auc:0.56448
[12]	train-auc:0.65426	eval-auc:0.56262
[13]	train-auc:0.66114	eval-auc:0.56407
[14]	train-auc:0.66519	eval-auc:0.56578
[15]	train-auc:0.67024	eval-auc:0.57594
[16]	train-auc:0.67229	eval-auc:0.57818
[17]	train-auc:0.67156	eval-auc:0.57375
[18]	train-auc:0.67424	eval-auc:0.57656
[19]	train-auc:0.67475	eval-auc:0.57856
[20]	train-auc:0.68278	eval-auc:0.58203
[21]	train-auc:0.68529	eval-auc:0.58409
[22]	train-auc:0.68368	eval-auc:0.58222
[23]	train-auc:0.68575	eval-auc:0.58242
[24]	train-auc:0.68590	eval-auc:0.58477
[25]	train

In [11]:
xgb_pred_label = list(bst.predict(x_perd))

In [12]:
lgbcallback = [early_stopping(300), log_evaluation(10)]
param = {
        'task': 'train',
        "max_depth": 5,
        'boosting_type': 'gbdt',  # 设置提升类型
        'objective': 'binary',  # 目标函数
        'metric': 'auc',  # 评估函数
        'learning_rate': 0.01,  # 学习速率
        # 建树的特征选择比例
        'feature_fraction': 1,
        'bagging_fraction': 0.7,  # 建树的样本采样比例
        'bagging_freq': 1,  # k 意味着每 k 次迭代执行bagging
        'min_data_in_leaf': 1,
        'verbose': -1,  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
        'device_type': 'gpu',
        'seed':666,
    }

dtrain = lgb.Dataset(X_train, y_train)
dtest = lgb.Dataset(X_test, y_test)
bst1 = lgb.train(param, dtrain, 3000, valid_sets=[
            dtest], callbacks=lgbcallback)

Training until validation scores don't improve for 300 rounds
[10]	valid_0's auc: 0.626177
[20]	valid_0's auc: 0.630181
[30]	valid_0's auc: 0.644048
[40]	valid_0's auc: 0.648723
[50]	valid_0's auc: 0.652761
[60]	valid_0's auc: 0.658387
[70]	valid_0's auc: 0.658766
[80]	valid_0's auc: 0.662132
[90]	valid_0's auc: 0.666741
[100]	valid_0's auc: 0.665946
[110]	valid_0's auc: 0.671887
[120]	valid_0's auc: 0.676457
[130]	valid_0's auc: 0.678
[140]	valid_0's auc: 0.680879
[150]	valid_0's auc: 0.685493
[160]	valid_0's auc: 0.687086
[170]	valid_0's auc: 0.688314
[180]	valid_0's auc: 0.690771
[190]	valid_0's auc: 0.691697
[200]	valid_0's auc: 0.691674
[210]	valid_0's auc: 0.693271
[220]	valid_0's auc: 0.694756
[230]	valid_0's auc: 0.695148
[240]	valid_0's auc: 0.697915
[250]	valid_0's auc: 0.698149
[260]	valid_0's auc: 0.698206
[270]	valid_0's auc: 0.700168
[280]	valid_0's auc: 0.701762
[290]	valid_0's auc: 0.702219
[300]	valid_0's auc: 0.702049
[310]	valid_0's auc: 0.704438
[320]	valid_0's auc:

In [13]:
lgb_pred_label=list(bst1.predict(df_test[x_col]))

In [14]:
import numpy as np

In [15]:
pred_labels=[lgb_pred_label,xgb_pred_label]

In [16]:
final_pred_label = np.exp(
    np.mean(np.log(pred_labels), axis=0))

In [17]:
lgb_pred_label

[0.012512210980609494,
 0.00502265309039638,
 0.019043220213105398,
 0.009747318781385173,
 0.02747927565058219,
 0.009666066710181694,
 0.023707245575614502,
 0.08593318065210268,
 0.026553118266270343,
 0.006839570699630944,
 0.007665399566817676,
 0.009266810192954816,
 0.031049713532937142,
 0.0029369487436459642,
 0.003493013424782191,
 0.04599933785637922,
 0.01421222011523043,
 0.08010907387696609,
 0.06458974890633143,
 0.031913765856082604,
 0.02923997890023014,
 0.047878847205380305,
 0.005552727664652684,
 0.011713767202908785,
 0.010394637699056132,
 0.3832063947108752,
 0.012555983357041754,
 0.0788625689989576,
 0.002347529206438213,
 0.0025935331197972844,
 0.01081061723512231,
 0.01450845281097225,
 0.07295434078931377,
 0.012127031000408498,
 0.00814260177183888,
 0.019868273861682974,
 0.052119412014660145,
 0.005495717891193291,
 0.004581760670017662,
 0.044560549715914125,
 0.06022361916772679,
 0.019881812672261058,
 0.0019281966486422461,
 0.015201174895440114,
 0

In [18]:
final_pred_label

array([0.01697862, 0.00489392, 0.01899141, ..., 0.04984891, 0.14101837,
       0.00722119])

In [19]:
import time

date = time.strftime("%Y%m%d-%H%M%S", time.localtime())

res = pd.DataFrame({
    'cust_id': list(df_test["cust_id"]),
    'label': lgb_pred_label,
})
res.to_csv("./output/result"+date+".csv", encoding="utf-8", index=False)